In [1]:
from datetime import datetime

import faiss
import numpy as np
import pandas as pd
import ruclip
import torch
from tqdm import tqdm
from transformers import (
	CLIPModel,
	CLIPConfig,
	CLIPTextConfig,
	CLIPVisionConfig,
	CLIPProcessor
)

# определяем девайс GPU/CPU
device = "cuda" if torch.cuda.is_available() else "cpu"

### загрузим ruCLIP
ru_clip, ru_clip_processor = ruclip.load('ruclip-vit-base-patch32-384', device="cpu")

# tqdm в режим pandas
tqdm.pandas()

C:\Users\HardWorkingStation\PyProjects\market_recommendations_lite\venv\lib\site-packages\huggingface_hub\file_download.py:594: FutureWarning: `cached_download` is the legacy way to download files from the HF hub, please consider upgrading to `hf_hub_download`
  warnings.warn(


In [2]:
f"Для подготовки данных будет использоваться: {'GPU' if device == 'cuda' else 'CPU'}"

'Для подготовки данных будет использоваться: GPU'

In [3]:
data = pd.read_csv("products.csv", index_col="sku")
# предположим, что не указан только отечественный производитель
data["country"] = data["country"].fillna("россия")
data.sample(10)

,id,name,brand,brand_type,dimension17,dimension18,dimension19,dimension20,country,price,...,main_product_sku,main_product_id,best_loyalty_price,dimension29,dimension28,description,product_usage,product_composition,category,category_ru
sku,,,,,,,,,,,,,,,,,,,,,
10015-19000013034,173649,skin illusion velvet,clarins,standard,тональный крем,женский,NaN,NaN,франция,3639,...,19000013034,173648,NaN,True,False,тональный крем с легкой текстурой моментально ...,наносите руками или специальной кистью для тон...,я создал clarins из любви и уважения к женской...,makijazh,макияж
82400800007,85183,french lover,frederic malle,middle,парфюмерная вода,мужской,NaN,NaN,франция,4530,...,82400800007,85183,4077.0,False,False,frederic malle french lover парфюмерная вода м...,frederic malle french lover парфюмерная вода м...,frederic malle french lover парфюмерная вода м...,parfjumerija,парфюмерия
100687-19000043104,193628,mystery,parisa cosmetics,standard,палетки для глаз,женский,NaN,NaN,испания,259,...,19000043104,193627,NaN,True,False,parisa cosmetics mystery палетки для глаз женс...,parisa cosmetics mystery палетки для глаз женс...,parisa cosmetics mystery палетки для глаз женс...,makijazh,макияж
19000021047,176493,pomegranate,hypno casa,standard,диффузоры,унисекс,NaN,NaN,италия,4680,...,19000021047,176493,3510.0,False,False,аромат распускается нежными нотами апельсина и...,предназначен для ароматизации воздуха в помеще...,парфюмерная композиция стекло ротанг картон hy...,parfjumerija,парфюмерия
90151400003,134630,"patchouli & jasmine, lemon",zielinski & rozen,middle,диффузоры,унисекс,NaN,NaN,израиль,8450,...,90151400003,134630,NaN,False,False,диффузор средство для ароматизации помещений в...,открыть ёмкость с ароматическим составом встав...,zielinski rozen артизанальный парфюмерный дом ...,parfjumerija,парфюмерия
19000022654,181733,агринион,nothing but love,standard,кольцо,унисекс,NaN,NaN,россия,909,...,19000022654,181733,818.0,False,False,nothing but love агринион кольцо унисекс кольц...,nothing but love агринион кольцо унисекс кольц...,nothing but love агринион кольцо унисекс кольц...,ukrashenija,украшения
82401300007,109880,lipstick rose,frederic malle,middle,гель для душа,женский,NaN,NaN,россия,3940,...,82401300007,109880,3546.0,False,False,frederic malle lipstick rose гель для душа жен...,frederic malle lipstick rose гель для душа жен...,frederic malle lipstick rose гель для душа жен...,parfjumerija,парфюмерия
19000051914,203624,№41 classic collection,dilis,standard,духи,женский,NaN,NaN,беларусь,2076,...,19000051914,203624,1868.0,False,False,семейство аромата шипровые цветочные начальные...,распылять на кожу с расстояния 15см не использ...,ароматическая композиция дистиллированная вода...,parfjumerija,парфюмерия
19000032324,186367,neroli,esteban paris parfums,standard,автопарфюм,унисекс,NaN,NaN,франция,1630,...,19000032324,186367,1467.0,False,False,теперь вы можете использовать свой любимый аро...,рекомендации по использованию и установке este...,керамика пластик картон esteban paris parfums ...,parfjumerija,парфюмерия


### Посчитаем эмбеддинги для текстового описания товаров

Сформируем столбец с описанием из разных колонок, по которому будем считать эмбеддинги

In [5]:
def get_full_desc(data_row):
	brand = data_row["brand"] if data_row["brand"] is not None else ""
	category_type = data_row["category_type"] if data_row["category_type"] is not None else ""
	country = data_row["country"] if data_row["country"] is not None else ""
	price = data_row["price"]
	dimension18 = data_row["dimension18"] if data_row["dimension18"] is not None else ""

	res = f"{category_type} {dimension18} {brand} {country} {price}"

	# ограничиваем длину описания до 2500 символов, чтобы мог отработать CLIP
	if len(res) > 2500:
		res = res[:2500]

	return res

data["full_desc"] = data.progress_apply(lambda row: get_full_desc(row), axis=1)

100%|██████████| 11843/11843 [00:00<00:00, 53103.24it/s]


Посчитаем эмбеддинги из ruCLIP для созданного описания товара

In [6]:
text_embeddings = pd.DataFrame()

for line in tqdm(data["full_desc"]):
	line_tokenized = ru_clip_processor(text=[line], return_tensors="pt", )
	with torch.no_grad():
		line_embedding = ru_clip.encode_text(line_tokenized["input_ids"]).to("cpu")
	text_embeddings = pd.concat([text_embeddings, pd.DataFrame(line_embedding)], ignore_index=True)

text_embeddings.index = data.index

  9%|▉         | 1104/11843 [00:40<06:36, 27.11it/s]


KeyboardInterrupt: 

Сохраним полученные эмбеддинги для текстового описания товаров

In [210]:
text_embeddings.to_csv("text_ruCLIP_embeddings_2023-01-25 3-00.csv")

Создадим FAISS index по полученным текстовым эмбеддингам и сохраним его

In [211]:
index = faiss.IndexFlatL2(text_embeddings.shape[1])
index.add(np.ascontiguousarray(text_embeddings.to_numpy().astype('float32')))
print(index.ntotal)  # теперь в нем n векторов
faiss.write_index(index, "text_ruCLIP_faiss.index")

11843


Потестируем рекомендации по полученному индексу

In [75]:
topn = 10
product_index_in_data = 2390
distances, same_embedding_indexes = index.search(np.ascontiguousarray(text_embeddings.to_numpy().astype('float32')[product_index_in_data].reshape((1, -1))), 10)
data.iloc[same_embedding_indexes[0]][["brand", "dimension18", "dimension19", "country", "price" , "category_type"]]

,brand,dimension18,dimension19,country,price,category_type
sku,,,,,,
19000042189,deep pink,унисекс,NaN,россия,159,морская розовая соль для ванн
19000034710,sel rose,унисекс,NaN,россия,120,крымская соль для ванн
19000034711,sel rose,для детей,NaN,россия,132,морская соль для ванн
19000051046,marespa,унисекс,NaN,россия,321,морская соль красного моря
19000021550,zielinski & rozen,унисекс,NaN,израиль,2880,соль для ванны
11975-19000021552,zielinski & rozen,унисекс,NaN,израиль,2880,соль для ванны
19000021549,zielinski & rozen,унисекс,NaN,израиль,2880,соль для ванны
19000021554,zielinski & rozen,унисекс,NaN,израиль,2070,соль для ванны
19000021553,zielinski & rozen,унисекс,NaN,израиль,2070,соль для ванны
